In [153]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score

In [22]:
data=pd.read_csv("hm_train.csv", header=0)
data.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category
0,27673,24h,I went on a successful date with someone I fel...,1,affection
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise
3,27676,24h,We had a serious talk with some friends of our...,2,bonding
4,27677,24h,I went with grandchildren to butterfly display...,1,affection


In [25]:
data=data[['cleaned_hm','predicted_category']]

In [59]:
from nltk.corpus import RegexpTokenizer as regextoken
stop = stopwords.words('english')
lmt = WordNetLemmatizer()
tokenizer = regextoken('\w+')

In [126]:
def process_text(text):
    sent = text.lower()
    word = word_tokenize(sent)
    clwrds = [w for w in word if not w in stop]
    lmtzr = WordNetLemmatizer()
    lmtzrs = [lmtzr.lemmatize(i) for i in clwrds]
    return(lmtzrs)

In [127]:
data['review_new']=data['cleaned_hm'].apply(process_text)

In [29]:
data['review_new'].head()

0    [went, successful, date, someone, felt, sympat...
1          [happy, son, got, 90, %, mark, examination]
2                        [went, gym, morning, yoga, .]
3    [serious, talk, friend, flaky, lately, ., unde...
4    [went, grandchild, butterfly, display, crohn, ...
Name: review_new, dtype: object

In [30]:
vectorizer=TfidfVectorizer()

In [31]:
data['review_clean'] = [' '.join(map(str, l)) for l in data['review_new']]

In [32]:
data['Review_processed'] = data['review_clean'].str.replace('[^\w\s]','')

In [34]:
review_drop_text=["review_clean","review_new"]

In [35]:
data.drop(review_drop_text,axis=1,inplace=True)

In [36]:
data.head()

,cleaned_hm,predicted_category,Review_processed
0,I went on a successful date with someone I fel...,affection,went successful date someone felt sympathy con...
1,I was happy when my son got 90% marks in his e...,affection,happy son got 90 mark examination
2,I went to the gym this morning and did yoga.,exercise,went gym morning yoga
3,We had a serious talk with some friends of our...,bonding,serious talk friend flaky lately understood g...
4,I went with grandchildren to butterfly display...,affection,went grandchild butterfly display crohn conser...


In [37]:
X = data['Review_processed']
y = data['predicted_category']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4,random_state=200)

In [131]:
vectorizer=TfidfVectorizer(max_features=200)
train_vectors=vectorizer.fit_transform(X_train)
validation_vectors=vectorizer.transform(X_test)
print(train_vectors.shape,validation_vectors.shape)


(36192, 200) (24129, 200)


In [132]:
rf=RandomForestClassifier()

In [135]:
rf.fit(train_vectors, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [136]:
pred=rf.predict(train_vectors)

In [137]:
pred_val=rf.predict(validation_vectors)

In [47]:
data_test=data_test['cleaned_hm']

In [138]:
data_test=pd.read_csv("hm_test.csv", header=0)

In [139]:
data_test.head()

,hmid,reflection_period,cleaned_hm,num_sentence
0,88305,3m,I spent the weekend in Chicago with my friends.,1
1,88306,3m,We moved back into our house after a remodel. ...,2
2,88307,3m,My fiance proposed to me in front of my family...,1
3,88308,3m,I ate lobster at a fancy restaurant with some ...,1
4,88309,3m,I went out to a nice restaurant on a date with...,5


In [140]:
data_test=data_test[['cleaned_hm']]

In [141]:
data_test.isnull().sum()

cleaned_hm    0
dtype: int64

In [144]:
def process_text(text):
    sent = text.lower()
    word = word_tokenize(sent)
    clwrds = [w for w in word if not w in stop]
    lmtzr = WordNetLemmatizer()
    lmtzrs = [lmtzr.lemmatize(i) for i in clwrds]
    return(lmtzrs)

In [145]:
data_test['review_new']=data_test['cleaned_hm'].apply(process_text)

In [146]:
data_test['review_clean'] = [' '.join(map(str, l)) for l in data_test['review_new']]

In [147]:
data_test['Review_processed'] = data_test['review_clean'].str.replace('[^\w\s]','')

In [148]:
a=data_test['Review_processed']

In [149]:
vectorizer.fit(a)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=200,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [150]:
test_vector=vectorizer.transform(a)

In [151]:
pred=rf.predict(test_vector)

In [157]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [160]:
confusion_matrix(X_test,pred_val)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [166]:
f1_score(y_test,pred_val, average='weighted')

0.7687078413404755